In [207]:
import pandas as pd


In [ ]:
data = pd.read_csv("test.csv", header=None)
data.head()

In [209]:
def clean_data(data):
    # rename columns
    data.columns = ['datetime', 'event']
    # filter data 
    data = data[(data.datetime != "time") | (data.event != "event") ]
    #data["event"] = data["event"].str.lower().str.replace(" ","_")
    data["event"] = data["event"].str.lower()

    # covert datetime column to datetime object. SHOULD BE DONE AFTER FILTERING. ELSE, ERRORS
    data.datetime = pd.to_datetime(data.datetime)
    data["date"] = data.datetime.dt.date
    data["month"] = data.datetime.dt.month_name()
    data['weekday'] = data.datetime.dt.day_name()
    data["time"] = data.datetime.dt.time   

    return data



In [210]:
classify_list = ["trip cct fault","station supply fail","dc power failure","cb lockout locked","cb backup trip",
"cb back prot trip","dc fault","diff comm fail fault", "cb err status","sf6 lockout", "relay fault"]
def classify_events(row):
    #print(row)  
    for s in classify_list:
        if s in str(row):
            return "critical"

    # mind the indentation. Else, the for loop might check only the first event in the "classify_list"
    else:
        return "normal"

In [ ]:
data = clean_data(data)
data.head()

In [ ]:

data['type'] = data['event'].apply(lambda row: classify_events(row))
data.head()

In [213]:
#critical_data.event.value_counts()

In [ ]:
data["substation_acronym"] = data["event"].apply(lambda x: x.split()[0])
data["fault_voltage"] = data["event"].apply(lambda x: x.split()[1])
data.head()

In [ ]:
critical_data = data[data.type == 'critical']
print(critical_data.shape)
critical_data.head()

In [216]:
# default: keep="first"
# substations = data.drop_duplicates(["substation_acronym"])
# substations.to_csv("substation_acronyms1.csv")

In [217]:
substations = data.drop_duplicates(subset=['substation_acronym'], keep='first')
substations['substation_acronym'].to_csv("substation_acronyms2.csv")

In [ ]:
excel_data = pd.read_excel("substation details.xlsx",sheet_name="substations", index_col='no')
excel_data.reset_index(inplace=True)
excel_data.head()

In [219]:
#data.to_csv("processed_data.csv")

## Combine dataframe. include substation details

In [ ]:
## read df
substation_data_columns = ['substation_acronym',"critical level","substation name"]
substation_data = pd.read_excel("substation details.xlsx", index_col="no",sheet_name=0)
final_data = data.merge(substation_data[substation_data_columns], how="inner", on="substation_acronym")
final_data.head()

In [223]:
final_data.to_csv("final_data.csv")